# backtrader - data feed

backtrader에 과거 주가 data를 등록하고 사용하는 방법에 대해 다루고 있습니다.

## 데이터 기간과 전략 실행기간 설정

전체 데이터 중에서 사용할 데이터 범위를 지정할 떄는 `fromdate` 와 `todate` 를 사용해 지정 할 수 있습니다.

In [31]:
import backtrader as bt
import datetime as dtm
from pyqqq.data.minutes import get_all_day_data

class St(bt.Strategy):
    def log(self, s):
        current_time = self.datas[0].datetime.datetime()
        print(f'{current_time}: {s}')

    def next(self):
        self.log(f'close={self.data.close[0]}')

fromdate = dtm.datetime(2024, 5, 23, 9, 30)
todate = dtm.datetime(2024, 5, 23, 15, 30)

dfs = get_all_day_data(fromdate.date(), ['005930'])
data = bt.feeds.PandasData(dataname=dfs['005930'], fromdate=fromdate, todate=todate)

cerebro = bt.Cerebro()
cerebro.adddata(data)
cerebro.addstrategy(St)
cerebro.run()

2024-05-23 09:30:00: close=77500.0
2024-05-23 09:31:00: close=77500.0
2024-05-23 09:32:00: close=77500.0
2024-05-23 09:33:00: close=77400.0
2024-05-23 09:34:00: close=77400.0
2024-05-23 09:35:00: close=77400.0
2024-05-23 09:36:00: close=77300.0
2024-05-23 09:37:00: close=77400.0
2024-05-23 09:38:00: close=77400.0
2024-05-23 09:39:00: close=77300.0
2024-05-23 09:40:00: close=77300.0
2024-05-23 09:41:00: close=77300.0
2024-05-23 09:42:00: close=77200.0
2024-05-23 09:43:00: close=77200.0
2024-05-23 09:44:00: close=77200.0
2024-05-23 09:45:00: close=77200.0
2024-05-23 09:46:00: close=77200.0
2024-05-23 09:47:00: close=77200.0
2024-05-23 09:48:00: close=77200.0
2024-05-23 09:49:00: close=77300.0
2024-05-23 09:50:00: close=77300.0
2024-05-23 09:51:00: close=77400.0
2024-05-23 09:52:00: close=77300.0
2024-05-23 09:53:00: close=77300.0
2024-05-23 09:54:00: close=77200.0
2024-05-23 09:55:00: close=77300.0
2024-05-23 09:56:00: close=77300.0
2024-05-23 09:57:00: close=77300.0
2024-05-23 09:58:00:

### Indicator 

이동평균선과 같은 지표를 사용하는 경우 해당 지표가 준비될 때까지는 거래가 일어나지 않습니다.

In [30]:
import backtrader as bt
import datetime as dtm
from pyqqq.data.minutes import get_all_day_data

class St(bt.Strategy):
    params = {
        'period': 60,
    }

    def __init__(self):
        self.sma = bt.indicators.SimpleMovingAverage(self.data, period=self.params.period)

    def log(self, s):
        current_time = self.datas[0].datetime.datetime()
        print(f'{current_time}: {s}')

    def next(self):
        self.log(f'close={self.data.close[0]} sma={self.sma[0]}')

dfs = get_all_day_data(fromdate.date(), ['005930'])
data = bt.feeds.PandasData(dataname=dfs['005930'])

cerebro = bt.Cerebro()
cerebro.adddata(data)
cerebro.addstrategy(St)
cerebro.run()

2024-05-23 09:59:00: close=77300.0 sma=77376.66666666667
2024-05-23 10:00:00: close=77300.0 sma=77370.0
2024-05-23 10:01:00: close=77200.0 sma=77358.33333333333
2024-05-23 10:02:00: close=77300.0 sma=77345.0
2024-05-23 10:03:00: close=77300.0 sma=77336.66666666667
2024-05-23 10:04:00: close=77300.0 sma=77330.0
2024-05-23 10:05:00: close=77400.0 sma=77330.0
2024-05-23 10:06:00: close=77300.0 sma=77326.66666666667
2024-05-23 10:07:00: close=77400.0 sma=77328.33333333333
2024-05-23 10:08:00: close=77300.0 sma=77328.33333333333
2024-05-23 10:09:00: close=77400.0 sma=77330.0
2024-05-23 10:10:00: close=77400.0 sma=77331.66666666667
2024-05-23 10:11:00: close=77200.0 sma=77330.0
2024-05-23 10:12:00: close=77300.0 sma=77330.0
2024-05-23 10:13:00: close=77300.0 sma=77328.33333333333
2024-05-23 10:14:00: close=77300.0 sma=77326.66666666667
2024-05-23 10:15:00: close=77300.0 sma=77325.0
2024-05-23 10:16:00: close=77200.0 sma=77321.66666666667
2024-05-23 10:17:00: close=77200.0 sma=77320.0
2024-05

### 거래 시작 시간 설정

따로 값을 지정하지 않아도 indicator의 값이 준비되기 전까지는 next가 호출되지 않습니다.
만약 실행 시점을 더 늦추고 싶다면 next() 함수에서 현재 시간을 보고 무시할 수 있습니다.

In [26]:
import backtrader as bt
import datetime as dtm
from pyqqq.data.minutes import get_all_day_data

class St(bt.Strategy):
    params = {
        'period': 60,
        'trading_start': dtm.time(9, 0),
    }

    def log(self, s):
        current_time = self.datas[0].datetime.datetime()
        print(f'{current_time}: {s}')

    def __init__(self):
        self.ma = bt.ind.SMA(self.data.close, period=self.params.period)

    def next(self):
        if self.datas[0].datetime.time() < self.params.trading_start:
            return

        self.log(f'close={self.data.close[0]}, ma={self.ma[0]}')

fromdate = dtm.datetime(2024, 5, 23, 9, 0)
todate = dtm.datetime(2024, 5, 23, 15, 30)

dfs = get_all_day_data(fromdate.date(), ['005930'])
data = bt.feeds.PandasData(dataname=dfs['005930'], fromdate=fromdate, todate=todate)

cerebro = bt.Cerebro()
cerebro.adddata(data)
cerebro.addstrategy(St, trading_start=dtm.time(11, 0)) # 11시부터 시작
cerebro.run()

fromdate = dtm.datetime(2024, 5, 23, 9, 0)
todate = dtm.datetime(2024, 5, 23, 15, 30)

dfs = get_all_day_data(fromdate.date(), ['005930'])
data = bt.feeds.PandasData(dataname=dfs['005930'], fromdate=fromdate, todate=todate)

cerebro = bt.Cerebro()
cerebro.adddata(data)
cerebro.addstrategy(St)
cerebro.broker.setcash(100000000)
cerebro.run()


2024-05-23 11:00:00: close=77300.0, ma=77283.33333333333
2024-05-23 11:01:00: close=77400.0, ma=77286.66666666667
2024-05-23 11:02:00: close=77400.0, ma=77288.33333333333
2024-05-23 11:03:00: close=77400.0, ma=77290.0
2024-05-23 11:04:00: close=77400.0, ma=77291.66666666667
2024-05-23 11:05:00: close=77500.0, ma=77293.33333333333
2024-05-23 11:06:00: close=77400.0, ma=77295.0
2024-05-23 11:07:00: close=77400.0, ma=77295.0
2024-05-23 11:08:00: close=77500.0, ma=77298.33333333333
2024-05-23 11:09:00: close=77500.0, ma=77300.0
2024-05-23 11:10:00: close=77500.0, ma=77301.66666666667
2024-05-23 11:11:00: close=77400.0, ma=77305.0
2024-05-23 11:12:00: close=77600.0, ma=77310.0
2024-05-23 11:13:00: close=77700.0, ma=77316.66666666667
2024-05-23 11:14:00: close=77700.0, ma=77323.33333333333
2024-05-23 11:15:00: close=77600.0, ma=77328.33333333333
2024-05-23 11:16:00: close=77600.0, ma=77335.0
2024-05-23 11:17:00: close=77600.0, ma=77341.66666666667
2024-05-23 11:18:00: close=77600.0, ma=77346

### 데이터 주기와 거래주기가 다른 경우 처리

분봉데이터를 사용하지만 1시간마다 전략 코드를 실행하고 싶은 경우에는 timer를 사용해서 처리할 수 있습니다.
- `add_timer()` : timer 등록
- `notify_timer()` : timer callback 
- Timer callback은 Indicator 준비 여부와 관계없이 호출됩니다.

호출되는 시점은 [문서 - When are timers called](https://www.backtrader.com/docu/timers/timers/#when-are-timers-called)를 참고하세요

In [44]:
import backtrader as bt
import datetime as dtm
import numpy as np
from pyqqq.data.minutes import get_all_day_data

class St(bt.Strategy):
    params = {
        'period': 60,
    }

    def __init__(self):
        self.add_timer(
            when=dtm.time(9, 20),
            repeat=dtm.timedelta(hours=1),
        )
        self.sma = bt.indicators.SimpleMovingAverage(self.data, period=self.params.period)

    def log(self, s):
        current_time = self.datas[0].datetime.datetime()
        print(f'{current_time}: {s}')

    def next(self):
        pass

    def notify_timer(self, timer, when, *args, **kwargs):
        self.log(f'close={self.data.close[0]} sma={self.sma[0]} when={when}')

dfs = get_all_day_data(fromdate.date(), ['005930'])
data = bt.feeds.PandasData(dataname=dfs['005930'])

cerebro = bt.Cerebro()
cerebro.adddata(data)
cerebro.addstrategy(St)
cerebro.run()

2024-05-23 09:20:00: close=77300.0 sma=nan when=2024-05-23 09:20:00
2024-05-23 10:20:00: close=77300.0 sma=77323.33333333333 when=2024-05-23 10:20:00
2024-05-23 11:20:00: close=77600.0 sma=77351.66666666667 when=2024-05-23 11:20:00
2024-05-23 12:20:00: close=78600.0 sma=78165.0 when=2024-05-23 12:20:00
2024-05-23 13:20:00: close=79000.0 sma=78776.66666666667 when=2024-05-23 13:20:00
2024-05-23 14:20:00: close=78800.0 sma=78666.66666666667 when=2024-05-23 14:20:00
2024-05-23 15:29:00: close=78300.0 sma=78678.33333333333 when=2024-05-23 15:20:00
